In [183]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from functools import partial

def generate_graph(size=10, nodes=10, edges=20, capacity=None):
    def filter_edge(capacity, g):
        graph = nx.to_networkx_graph(g).edges()
        def filter_func(pair, graph):
            ed = set(graph)
            k, _ = pair
            if k in ed:
                return True
            else:
                return False
        filter_f = partial(filter_func, graph=graph)
        return dict(filter(filter_f, capacity.items()))

    if capacity is None:
        capacity = {}
        G = nx.complete_graph(nodes)
        for edge in G.edges:
            capacity[edge] =  randint(60, 300)
    graph = np.array([nx.to_numpy_array(nx.gnm_random_graph(n=nodes, m=edges)) for _ in range(size)])
    graph_cap = np.array([list(filter_edge(capacity, G).values()) for G in graph])
    return graph, graph_cap

In [ ]:
from numpy.random import choice, randint, random

def clone_ind(size, population, pCl):
    idx = choice(a=size, size = int(np.ceil(pCl*size)), replace=False)
    return idx

In [ ]:
def mutate_outga(individual, pMu=0.2):
    if random() > pMu:
        return individual
    G = nx.to_networkx_graph(individual)
    edges = list(G.edges())
    n_nodes = G.number_of_nodes()
    edge = edges[choice(a=len(edges))]
    s, t = edge
    exclusion = [s, t]
    source = choice(exclusion)
    for s, t in edges:
        if s == source:
            exclusion.append(t)
        elif t == source:
            exclusion.append(s)

    i = choice(a=[i for i in range(n_nodes) if i not in exclusion ])
    ind = individual.copy()
    ind[s][t] = 0
    ind[t][s] = 0

    ind[source][i] = 1
    ind[i][source] = 1

    return ind


In [ ]:
def crossover_outga(parent1, parent2, edge_size,pCr=0.8):
    if random() > pCr:
        return False
    p1, p2 = parent1.copy(), parent2.copy()
    chrom_len = len(p1)
    temp = (p1 + p2) / 2

    redundant = np.floor(temp)
    new_pair = np.ceil(temp - redundant)
    total_pair = redundant + new_pair

    redundant_sum = redundant.sum()/2
    np_sum = new_pair.sum()/2
    current_sum = redundant_sum + np_sum

    if current_sum > edge_size:
        nr_edge = int(current_sum - edge_size)
        if nr_edge == redundant_sum:
            return new_pair
        elif nr_edge < redundant_sum:
            G = nx.to_networkx_graph(redundant)
            edges = np.array(G.edges())
            print(nr_edge)
            edge = edges[choice(a=len(edges), size=nr_edge,replace=False)]
            for s, t in edge:
                redundant[s][t] = 0
                redundant[t][s] = 0
            return redundant + new_pair
        elif nr_edge > redundant_sum:
            G = nx.to_networkx_graph(total_pair)
            edges = np.array(G.edges())
            edge = edges[choice(a=len(edges), size=nr_edge,replace=False)]
            for s, t in edge:
                total_pair[s][t] = 0
                total_pair[t][s] = 0
            return total_pair


In [165]:
def fitness_outga(individual, flow):
    G = nx.to_networkx_graph(individual)
    min_val, max_val = np.min(flow), np.max(flow)
    scaled = (flow - min_val) / (max_val - min_val)
    for (i, (u,v)) in enumerate(G.edges):
        G[u][v]['flow'] = scaled[i]
    T = nx.average_shortest_path_length(G, weight="flow", method='dijkstra')
    return 1/T
    

In [ ]:
def tournament_selection(size: int, pop: np.array, fitness: np.array, k=5):

    index = []
    population = list(pop)
    select = []
    for _ in range(size):
        selection_ix = randint(len(population))
        for ix in np.random.randint(0, len(population), k-1):
            if fitness[ix] > fitness[selection_ix]:
                selection_ix = ix
        select.append(population.pop(selection_ix))
        index.append(selection_ix)

    return np.array(select), np.array(index)

In [180]:
flow = [225.60893090881635, 31.14703832125048, 69.67571471085778, 69.97231021118779, 128.34167952772043, 304.0239049986733, 133.50762482856462, 124.78672295159305, 52.10243245710136, 54.85662776475171, 43.9049797566212, 135.37255182191663, 88.7276133744623, 55.84638578480374, 96.94228446751892, 50.50591520227935, 123.34045189153773, 46.062823976262045, 22.287162849768496, 99.8143759994609]
path = generate_graph()[0]

In [181]:
fitness_outga(path, flow)

3.9459579068172013